In [5]:

from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
import os
from dotenv import load_dotenv
load_dotenv()  # Load environment variables from .env file
llm = ChatGroq(model="openai/gpt-oss-120b", temperature=0, api_key=os.getenv("GROQ_API_KEY"))   # Initialize LLM with specific model and temperature
prompt1 = ChatPromptTemplate.from_messages([
    ("system","you are a strict {subject} tutor."),    # System message with a placeholder
    ("human","Explain {topic} in {difficulty} terms.")     # Human message with placeholders
])
prompt2 = ChatPromptTemplate.from_messages([
    ("human","You are a strict {subject} teacher"),    # Human message with a placeholder
    ("human","Explain {topic} in {difficulty} terms.")     # Human message with placeholders
])
message1 = prompt1.format_prompt(subject="math", topic="calculus", difficulty="beginner")
message2 = prompt2.format_prompt(subject="math", topic="calculus", difficulty="beginner")

message1 = message1.to_messages()
message2 = message2.to_messages()

print("Message 1:")
for msg in message1:
    print(f"{msg.type}: {msg.content}")
print("\nMessage 2:")
for msg in message2:
    print(f"{msg.type}: {msg.content}")

response1 = llm.invoke(message1)
response2 = llm.invoke(message2)
print("Response to Prompt 1:")
print(response1.content)    
print("\nResponse to Prompt 2:")
print(response2.content)    

Message 1:
system: you are a strict math tutor.
human: Explain calculus in beginner terms.

Message 2:
human: You are a strict math teacher
human: Explain calculus in beginner terms.
Response to Prompt 1:
## Calculus in Beginner Terms  
*(A concise, step‑by‑step guide from a strict‑but‑supportive math tutor)*  

---

### 1. What Is Calculus, Really?
Calculus is the branch of mathematics that studies **change** and **accumulation**.  
- **Change** → How something varies moment‑to‑moment (think speed, slope, growth).  
- **Accumulation** → How small pieces add up to a whole (think distance traveled, area under a curve).

We handle these ideas with two main tools:

| Tool | What It Measures | Everyday Analogy |
|------|------------------|------------------|
| **Derivative** | Instantaneous rate of change (the “slope”) | How fast a car’s speedometer reads at an exact instant |
| **Integral** | Total accumulation of tiny pieces | How far the car travels after a certain amount of time |

---

In [8]:
# simulate conversation
messages = [
    SystemMessage(content="You are a very strict math tutor."),     # System message to set the context
    HumanMessage(content="Explain gradient descent in one paragraph."),      # Human message with the user's query
]
response = llm.invoke(messages)
print("\nConversation Response:")
print(response.content)
messages.append(AIMessage(content=response.content))
messages.append(HumanMessage(content="Can you provide a simple example?"))
response = llm.invoke(messages)
print("\nFollow-up Response:")
print(response.content)


Conversation Response:
Gradient descent is an iterative optimization algorithm used to find a local minimum of a differentiable function \(f(\mathbf{x})\); starting from an initial guess \(\mathbf{x}_0\), it updates the parameters by moving them in the direction of the steepest decrease, namely the negative gradient, according to the rule \(\mathbf{x}_{k+1} = \mathbf{x}_k - \alpha_k \nabla f(\mathbf{x}_k)\), where \(\alpha_k>0\) is a step‑size (or learning rate) that may be constant or adaptively chosen; each iteration reduces the function value (provided \(\alpha_k\) is suitably small), and under appropriate conditions—convexity of \(f\) and a diminishing step‑size satisfying \(\sum_k \alpha_k = \infty\) and \(\sum_k \alpha_k^2 < \infty\)—the sequence \(\{\mathbf{x}_k\}\) converges to a global minimizer, while for non‑convex functions it converges to a critical point (which may be a local minimum, saddle point, or maximum).

Follow-up Response:
**Example: Minimising a one‑dimensional

In [11]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

from langchain.prompts import HumanMessagePromptTemplate, SystemMessagePromptTemplate
Human_message = HumanMessagePromptTemplate.from_template("Explain {topic} in {difficulty} terms.",input_variables=["topic","difficulty"])
System_message = SystemMessagePromptTemplate.from_template("You are a strict {subject} tutor.",input_variables=["subject"])


prompt = ChatPromptTemplate.from_messages([System_message,Human_message])

print("\nChained Prompt Messages:")
for msg in prompt.format_prompt(subject="math",topic="calculus",difficulty="beginner").to_messages():
    print(f"{msg.type}: {msg.content}")

# Create a chain that maps inputs to the prompt, then to the LLM, and finally extracts the AI response


chain = (
    {"subject": lambda x : x["subject"], "topic": lambda x : x["topic"], "difficulty":lambda x : x["difficulty"]} |
    prompt |
    llm |
    StrOutputParser()
)
final_response = chain.invoke({"subject":"math","topic":"calculus","difficulty":"beginner"})



Chained Prompt Messages:
system: You are a strict math tutor.
human: Explain calculus in beginner terms.


In [12]:
print(final_response)

**Calculus – The Very Basics (Explained Strictly and Clearly)**  

---

### 1. What Calculus Is **and** What It Is Not  

- **Calculus** is the branch of mathematics that studies **change** and **accumulation**.  
- It is **not** a magic trick that solves every problem instantly. You must understand the *why* before you can apply the *how*.  

---

### 2. Two Core Ideas  

| Idea | What It Measures | Symbol (most common) | Everyday Analogy |
|------|------------------|----------------------|------------------|
| **Derivative** | Instantaneous rate of change (how fast something is changing at a single point) | \(f'(x)\) or \(\dfrac{dy}{dx}\) | Speedometer: tells you your speed **right now**, not the average speed over a trip. |
| **Integral** | Accumulated quantity (area under a curve, total amount accumulated) | \(\displaystyle\int f(x)\,dx\) | Odometer: adds up every tiny bit of distance you travel to give the total miles. |

You must master **limits** first; they are the foundation o